## GTFS - Transfer Edges

In [169]:
import bisect
import pandas as pd
import datetime
import numpy as np
import geopy.distance
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
tqdm.pandas()
import dask.dataframe as dd
from dask.multiprocessing import get
from geopy.point import Point
import datetime as dt
from multiprocessing import Pool, cpu_count
from contextlib import closing
import math

from dask.diagnostics import ProgressBar
ProgressBar().register()

DATA_PATH = '../../../input_data/synthetic_examples/input_data/Test1/'

In [170]:
OUTPUT_PATH = '../../../output_data/validation/two_lines/'

In [171]:
DAY = dt.datetime(2015, 12, 14)

## Load Nodes

In [172]:
# Load nodes
FILES_PREFIX = 'morning'
file_name = OUTPUT_PATH + FILES_PREFIX + '_nodes.pkl'
print(f'Loading nodes from file {file_name}')
nodes_df = pd.read_pickle(file_name)
nodes_df.head(3)

Loading nodes from file ../../../output_data/validation/two_lines/morning_nodes.pkl


,index,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
0,1,19407596_081215,07:06:06,07:06:06,12962,6,2544,06:59:00,21257,32.076767,34.844303,2015-12-14 07:06:06,2015-12-14 07:06:06,0
1,2,19407597_081215,07:15:06,07:15:06,12962,6,2544,07:08:00,21257,32.076767,34.844303,2015-12-14 07:15:06,2015-12-14 07:15:06,1
2,3,19407598_081215,07:24:06,07:24:06,12962,6,2544,07:17:00,21257,32.076767,34.844303,2015-12-14 07:24:06,2015-12-14 07:24:06,2


In [173]:
nodes_df.shape

(608, 14)

In [174]:
nodes_df.nunique()

index                            608
trip_id                           26
arrival_time                     573
departure_time_stop              573
stop_id                           64
stop_sequence                     37
route_id                           2
departure_time_trip_departure     23
stop_code                         64
stop_lat                          64
stop_lon                          64
arrival                          573
departure                        573
node_id                          608
dtype: int64

### Filter nodes for development

In [175]:
# start_time = DAY + dt.timedelta(hours=8)
# end_time = start_time + dt.timedelta(minutes=10)

# nodes_df = nodes_df[nodes_df['arrival'] > start_time][nodes_df['arrival'] < end_time]

In [176]:
# nodes_df.shape

In [177]:
# nodes_df.nunique()

## Parallelizing

In [178]:
def run_parallel_df(df_grouped, func):
    with closing(Pool()) as pool:
        res_list = pool.map(func, [group for name, group in df_grouped])
    return pd.concat(res_list)

In [179]:
def run_parallel_dict(data, num_of_batches, func):
    batch_size = math.ceil(len(data) / num_of_batches)
    with closing(Pool()) as pool:
#         res_list = []
        data_items = list(data.items())
        print(f'Processing data in workers pool. Numner of batches is {num_of_batches} where each batch has approx. {batch_size} keys')
        sliced_data = [data_items[i:i+batch_size] for i in range(num_of_batches)]
        print('Finished slicing the data.')
#         curr_batch = 1
        res_list = pool.map(func, sliced_data)
#         for res in tqdm_notebook(pool.imap(func, sliced_data), total=num_of_batches):
#             res_list.append(res)
#             print(f'Finished processing batch number {curr_batch} of {num_of_batches} total. Number of transfers from this batch: {len(res)}')
#             curr_batch += 1
    return [item for sublist in res_list for item in sublist]

#### Testing parallelization

In [180]:
# d = {'a': [1,2,3], 'b':[1]}
# # num_of_batches = 2
# # d_items = list(d.items())
# # batch_size = math.ceil(len(d) / num_of_batches)
# # sliced_d = [d_items[i:i+batch_size] for i in range(num_of_batches)]
# # print(len(d))

# def print_dict_with_sleep(data):
#     print('processing..')
#     data = dict(data)
#     for k,v in data.items():
#         print(len(v))
# #     time.sleep(3)
# run_parallel_dict(d, 2, print_dict_with_sleep)

## Load Nearby Stops

In [213]:
stops_mapping_df = pd.read_csv(DATA_PATH + 'StopsToStops.csv')

In [214]:
stops_mapping_df.head(3)

,in_stop_id,near_stop_id,near_dist
0,12834,12834,0.0
1,12841,12841,0.0
2,12889,12889,0.0


In [215]:
stops_mapping_df.columns 

Index(['in_stop_id', 'near_stop_id', 'near_dist'], dtype='object')

In [216]:
stops_mapping_df.columns = ['IN_FID', 'NEAR_FID', 'NEAR_DIST']

In [217]:
# stops_mapping_df.shape

In [218]:
# stops_mapping_df.nunique()

If we allow walking 300 meters we get an average of 6.24 transfers to neighboring stops from each stop. 

In [219]:
# TODO: 
# change walk speed to 1m/s - IMPL
# maybe switch to 100-150
# 15 minutes walk + wait

# hoping to get to 10-20 edges per node

In [220]:
stops_df = pd.read_csv(DATA_PATH + 'stops.txt')

### Get all stops from which we wish to start computations

In [221]:
nodes_df.head()

,index,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
0,1,19407596_081215,07:06:06,07:06:06,12962,6,2544,06:59:00,21257,32.076767,34.844303,2015-12-14 07:06:06,2015-12-14 07:06:06,0
1,2,19407597_081215,07:15:06,07:15:06,12962,6,2544,07:08:00,21257,32.076767,34.844303,2015-12-14 07:15:06,2015-12-14 07:15:06,1
2,3,19407598_081215,07:24:06,07:24:06,12962,6,2544,07:17:00,21257,32.076767,34.844303,2015-12-14 07:24:06,2015-12-14 07:24:06,2
3,4,19407599_081215,07:33:06,07:33:06,12962,6,2544,07:26:00,21257,32.076767,34.844303,2015-12-14 07:33:06,2015-12-14 07:33:06,3
4,5,19407600_081215,07:42:06,07:42:06,12962,6,2544,07:35:00,21257,32.076767,34.844303,2015-12-14 07:42:06,2015-12-14 07:42:06,4


In [222]:
t1 = DAY+dt.timedelta(hours=7, minutes=15)
t2 = t1 + dt.timedelta(minutes=15)
start_nodes_df = nodes_df[nodes_df['departure'] <= t2][nodes_df['departure'] >= t1]

d:\users\bensdani\venvs\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [223]:
start_nodes_df.shape

(107, 14)

In [224]:
start_nodes_df.head()

,index,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
1,2,19407597_081215,07:15:06,07:15:06,12962,6,2544,07:08:00,21257,32.076767,34.844303,2015-12-14 07:15:06,2015-12-14 07:15:06,1
2,3,19407598_081215,07:24:06,07:24:06,12962,6,2544,07:17:00,21257,32.076767,34.844303,2015-12-14 07:24:06,2015-12-14 07:24:06,2
10,71,19407597_081215,07:16:20,07:16:20,12979,7,2544,07:08:00,21280,32.082794,34.845456,2015-12-14 07:16:20,2015-12-14 07:16:20,10
11,72,19407598_081215,07:25:20,07:25:20,12979,7,2544,07:17:00,21280,32.082794,34.845456,2015-12-14 07:25:20,2015-12-14 07:25:20,11
19,140,19407597_081215,07:17:38,07:17:38,13191,8,2544,07:08:00,21631,32.092032,34.847151,2015-12-14 07:17:38,2015-12-14 07:17:38,19


In [225]:
start_nodes_list = start_nodes_df['node_id'].to_list()
start_nodes_list = [str(n) for n in start_nodes_list]

In [226]:
with open((OUTPUT_PATH + FILES_PREFIX + '_start_nodes.pkl'), 'wb') as f:
    pickle.dump(start_nodes_list, f)

#### Validation - let's find the most frequent origin stops and make sure it seems right.

In [227]:
# stops_mapping_150m_df.IN_FID.mode()

In [228]:
# stops_df.iloc[10841]

In [229]:
# stops_df.iloc[10841]

We can see this is the central bus station, which makes sense it will have the most possible transfers.

Now we must re-map the nearby stops to stop_id as used in the GTFS feeds.

In [230]:
stops_df['IN_FID'] = stops_df['stop_id']

In [231]:
stops_df[['IN_FID', 'stop_id']].head()

,IN_FID,stop_id
0,12834,12834
1,12841,12841
2,12889,12889
3,12893,12893
4,12894,12894


In [232]:
stops_df.shape

(64, 9)

In [233]:
stops_mapping_df.head(2)

,IN_FID,NEAR_FID,NEAR_DIST
0,12834,12834,0.0
1,12841,12841,0.0


In [234]:
stops_mapping_df = stops_mapping_df.merge(stops_df[['IN_FID', 'stop_id']], on='IN_FID', how='left')
stops_mapping_df.rename(columns={'stop_id':'from_stop_id'}, inplace=True)
stops_mapping_df = stops_mapping_df.merge(stops_df[['IN_FID', 'stop_id']], left_on='NEAR_FID', right_on='IN_FID', how='left')
stops_mapping_df.rename(columns={'stop_id':'to_stop_id', 'NEAR_DIST':'dist'}, inplace=True)

In [235]:
stops_mapping_df = stops_mapping_df[['from_stop_id', 'to_stop_id', 'dist']]
stops_mapping_df.head()

,from_stop_id,to_stop_id,dist
0,12834,12834,0.0
1,12841,12841,0.0
2,12889,12889,0.0
3,12893,12893,0.0
4,12894,12894,0.0


In [239]:
AVG_WALK_SPEED = 0.83  # meters per second (m/s)
stops_mapping_df['walk_time_sec'] = stops_mapping_df['dist'] / AVG_WALK_SPEED

In [240]:
stops_mapping_df.head(3)

,from_stop_id,to_stop_id,dist,walk_time_sec
0,12834,12834,0.0,0.0
1,12841,12841,0.0,0.0
2,12889,12889,0.0,0.0


In [241]:
stops_mapping_df.to_csv(OUTPUT_PATH + 'nearby_stops.csv')

In [242]:
stops_mapping_df.to_pickle(OUTPUT_PATH + 'nearby_stops.pkl')

In [243]:
stops_mapping_df.shape

(72, 4)

In [244]:
stops_mapping_df.nunique()

from_stop_id     64
to_stop_id       64
dist              5
walk_time_sec     5
dtype: int64

In [245]:
# stops_mapping_df = pd.read_pickle(OUTPUT_PATH + 'nearby_stops.pkl')

In [246]:
stops_mapping_200m_df = stops_mapping_df[stops_mapping_df['dist'] < 200]

In [247]:
stops_mapping_200m_df.shape

(72, 4)

In [248]:
stops_mapping_200m_df.nunique()

from_stop_id     64
to_stop_id       64
dist              5
walk_time_sec     5
dtype: int64

## Compute Transfer Edges

In [249]:
stops_to_nodes = nodes_df.groupby('stop_id')[[
    'node_id', 'trip_id', 'arrival', 'departure', 'route_id']].apply(lambda g: g.values.tolist()).to_dict()

In [250]:
dict(list(stops_to_nodes.items())[0:2])

{12834: [[599,
   '19404968_091215',
   Timestamp('2015-12-14 07:10:45'),
   Timestamp('2015-12-14 07:10:45'),
   9807],
  [600,
   '19404969_091215',
   Timestamp('2015-12-14 07:20:45'),
   Timestamp('2015-12-14 07:20:45'),
   9807],
  [601,
   '19404970_091215',
   Timestamp('2015-12-14 07:28:45'),
   Timestamp('2015-12-14 07:28:45'),
   9807],
  [602,
   '19404971_091215',
   Timestamp('2015-12-14 07:36:45'),
   Timestamp('2015-12-14 07:36:45'),
   9807],
  [603,
   '19404972_091215',
   Timestamp('2015-12-14 07:44:45'),
   Timestamp('2015-12-14 07:44:45'),
   9807],
  [604,
   '19404973_091215',
   Timestamp('2015-12-14 07:53:45'),
   Timestamp('2015-12-14 07:53:45'),
   9807],
  [605,
   '19404974_091215',
   Timestamp('2015-12-14 08:03:45'),
   Timestamp('2015-12-14 08:03:45'),
   9807],
  [606,
   '19404975_091215',
   Timestamp('2015-12-14 08:13:45'),
   Timestamp('2015-12-14 08:13:45'),
   9807],
  [607,
   '19404976_091215',
   Timestamp('2015-12-14 08:24:45'),
   Timestamp('

In [251]:
dict(list(stops_to_nodes.items()))

{12834: [[599,
   '19404968_091215',
   Timestamp('2015-12-14 07:10:45'),
   Timestamp('2015-12-14 07:10:45'),
   9807],
  [600,
   '19404969_091215',
   Timestamp('2015-12-14 07:20:45'),
   Timestamp('2015-12-14 07:20:45'),
   9807],
  [601,
   '19404970_091215',
   Timestamp('2015-12-14 07:28:45'),
   Timestamp('2015-12-14 07:28:45'),
   9807],
  [602,
   '19404971_091215',
   Timestamp('2015-12-14 07:36:45'),
   Timestamp('2015-12-14 07:36:45'),
   9807],
  [603,
   '19404972_091215',
   Timestamp('2015-12-14 07:44:45'),
   Timestamp('2015-12-14 07:44:45'),
   9807],
  [604,
   '19404973_091215',
   Timestamp('2015-12-14 07:53:45'),
   Timestamp('2015-12-14 07:53:45'),
   9807],
  [605,
   '19404974_091215',
   Timestamp('2015-12-14 08:03:45'),
   Timestamp('2015-12-14 08:03:45'),
   9807],
  [606,
   '19404975_091215',
   Timestamp('2015-12-14 08:13:45'),
   Timestamp('2015-12-14 08:13:45'),
   9807],
  [607,
   '19404976_091215',
   Timestamp('2015-12-14 08:24:45'),
   Timestamp('

In [252]:
DEPARTURE_INDEX = 3
for stop, nodes in stops_to_nodes.items():
    stops_to_nodes[stop] = sorted(nodes, key=lambda x: x[DEPARTURE_INDEX])

In [253]:
total_values = 0
max_nodes_in_stop = 0
for n in stops_to_nodes.values():
    num_nodes = len(n)
    if num_nodes > max_nodes_in_stop:
        max_nodes_in_stop = num_nodes
    total_values += num_nodes
print(f'There is a total of {total_values} nodes in the stops_to_nodes dictionary')

There is a total of 608 nodes in the stops_to_nodes dictionary


In [254]:
len(stops_to_nodes)

64

In [255]:
print(f'We have an average of {total_values/len(stops_to_nodes)}, and a max of {max_nodes_in_stop} nodes in a single stop.')

We have an average of 9.5, and a max of 19 nodes in a single stop.


In [256]:
ARRIVAL_INDEX = 2
DEPARTURE_INDEX = 3
ROUTE_ID_INDEX = 4
MAX_WAIT_TIME = dt.timedelta(minutes=10)
def get_transfer_edges_from_nodes(stops_to_nodes_batch):
    print('Processing batch...')
    stops_to_nodes_batch = dict(stops_to_nodes_batch)
    transfer_edges = []
    for stop, nodes in tqdm(stops_to_nodes_batch.items()):
#         print(f'Handling stop: {stop}')
        for start_n in nodes:
#             print(f'Start node: {start_n}')
            nearby_stops_df = stops_mapping_200m_df[stops_mapping_200m_df['from_stop_id'] == stop]
            # Add current stop to check transfers from the same stop
            nearby_stops_df.append({'from_stop_id': [stop], 'to_stop_id': [stop], 'dist': [0], 'walk_time_sec': [0]}, ignore_index=True)
            # TODO: verify this we're not staying to the same line in same direction
#             print(f'Going though nearby stops')
            for s in nearby_stops_df.iterrows():
                nearby_stop_id = s[1]['to_stop_id']
#                 print(f'Nearby stop: {nearby_stop_id}')
                if nearby_stop_id not in stops_to_nodes:
                    # Some stops don't have trips that operate all week. Some operate only on weekends.
                    # If this is such a stop we should continue to look at other stops, we won't find any
                    # nodes here.
                    continue
                nearby_nodes = stops_to_nodes[nearby_stop_id]
                second_line_earliest_start_time = start_n[ARRIVAL_INDEX] + dt.timedelta(seconds=s[1]['walk_time_sec'])
                second_line_latest_start_time = second_line_earliest_start_time + MAX_WAIT_TIME
                # Find index of first node that departs at least at second_start_time or later
                _, _, _, departures, _ = zip(*nearby_nodes)
                i = bisect.bisect_left(departures, second_line_earliest_start_time)
#                 print(f'Number of nodes in current stop: {len(nearby_nodes)}')
                while (i < len(nearby_nodes) and 
                       nearby_nodes[i][DEPARTURE_INDEX] >= second_line_earliest_start_time and 
                       nearby_nodes[i][DEPARTURE_INDEX] <= second_line_latest_start_time):
#                     print(f'Handling node with index {i}: {nearby_nodes[i]}')
                    node = nearby_nodes[i][0]
                    if node == start_n[0] or (nearby_nodes[i][ROUTE_ID_INDEX] == start_n[ROUTE_ID_INDEX] and s == stop):
                        # We don't wish to transfer to the same node (no self-edges). 
                        # Another case we wish to avoid is transferring to the same line (route__id) in the same stop.
                        i += 1
                        continue
                    transfer_edges.append((start_n[0], nearby_nodes[i][0], nearby_nodes[i][DEPARTURE_INDEX] - start_n[ARRIVAL_INDEX]))
                    i += 1
    return transfer_edges

In [257]:
# transfer_edges = run_parallel_dict(stops_to_nodes, 1000, get_transfer_edges_from_nodes)

In [258]:
transfer_edges = get_transfer_edges_from_nodes(stops_to_nodes)

Processing batch...


In [259]:
len(transfer_edges)

550

In [260]:
transfer_edges[:3]

[(599, 600, Timedelta('0 days 00:10:00')),
 (600, 601, Timedelta('0 days 00:08:00')),
 (601, 602, Timedelta('0 days 00:08:00'))]

In [68]:
nodes_df[nodes_df['node_id'] == 9]

,index,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
9,90,19415585_091215,07:12:00,07:12:00,12936,2,9823,07:10:00,21227,32.106707,34.802776,2015-12-14 07:12:00,2015-12-14 07:12:00,9


In [69]:
nodes_df[nodes_df['node_id'] == 63]

,index,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
63,639,19415585_091215,07:20:01,07:20:01,13950,11,9823,07:10:00,26315,32.106109,34.8045,2015-12-14 07:20:01,2015-12-14 07:20:01,63


## TODO: we should eliminate transfers of the same trip_id

In [261]:
transfer_edges_sec = []
for s, t, w in transfer_edges:
    transfer_edges_sec.append((s, t, w.total_seconds()))

In [262]:
transfer_edges_sec[0:5]

[(599, 600, 600.0),
 (600, 601, 480.0),
 (601, 602, 480.0),
 (602, 603, 480.0),
 (603, 604, 540.0)]

In [263]:
import pickle
with open(OUTPUT_PATH + FILES_PREFIX + '_transfer_edges.pkl', 'wb') as f:
    print(f'Saving transfers to file {f.name}')
    pickle.dump(transfer_edges_sec, f)

Saving transfers to file ../../../output_data/validation/two_lines/morning_transfer_edges.pkl


## Let's check some transfers manually

In [46]:
transfer_edges[1000:1005]

[(1120240, 3243432, Timedelta('0 days 00:06:00')),
 (1120240, 3243400, Timedelta('0 days 00:14:04')),
 (1120240, 3243469, Timedelta('0 days 00:11:49')),
 (1120240, 1119942, Timedelta('0 days 00:09:52')),
 (1120240, 1119886, Timedelta('0 days 00:12:17'))]

In [47]:
nodes_df[nodes_df['node_id'] == 3243432]

,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
3243432,36877179_271019,08:41:49,08:41:49,38971,39,25434,ראשון לציון_רכבת הראשונים,07:50:00,30064,31.947781,34.823278,2019-11-03 08:41:49,2019-11-03 08:41:49,3243432


In [48]:
nodes_df[nodes_df['node_id'] == 1120240]

,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
1120240,42245728_271019,08:35:49,08:35:49,76,53,25581,רכבת הראשונים,07:40:00,38906,31.948552,34.822422,2019-11-03 08:35:49,2019-11-03 08:35:49,1120240


In [49]:
nodes_df[nodes_df['node_id'] == 3243400]

,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
3243400,36873380_271019,08:49:53,08:49:53,38971,31,25405,ראשון לציון_רכבת הראשונים,08:10:00,30064,31.947781,34.823278,2019-11-03 08:49:53,2019-11-03 08:49:53,3243400


In [50]:
transfer_edges[4000000:4000002]

[(1763216, 3026510, Timedelta('0 days 00:09:05')),
 (1763216, 3026503, Timedelta('0 days 00:14:05'))]

In [51]:
nodes_df[nodes_df['node_id'] == 3026510]

,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
3026510,42299653_011119,09:00:00,09:00:00,42875,1,51,אשדוד_תחנה מרכזית,09:00:00,20001,32.083157,34.796091,2019-11-03 09:00:00,2019-11-03 09:00:00,3026510


In [52]:
nodes_df[nodes_df['node_id'] == 1763216]

,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
1763216,36873382_271019,08:50:55,08:50:55,38946,2,25405,ראשון לציון_רכבת הראשונים,08:50:00,20015,32.081706,34.795561,2019-11-03 08:50:55,2019-11-03 08:50:55,1763216


In [53]:
nodes_df[nodes_df['node_id'] == 3026503]

,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
3026503,35771757_271019,09:05:00,09:05:00,42875,1,20871,ראש העין_פארק תעשיה אפק,09:05:00,20001,32.083157,34.796091,2019-11-03 09:05:00,2019-11-03 09:05:00,3026503
